# Setup

In [ ]:
!git clone git@github.com:hnoya/prj-kaggle-retrospection.git

%cd prj-kaggle-retrospection/2022-NBME

- 次のセルの`$YOUR_JSON_PATH`は自身のkaggle.jsonのパスに置き換える
    - e.g. `"/content/drive/MyDrive/kaggle.json"` など

In [ ]:
#!pip install -q kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle

!mkdir ~/.kaggle
!cp $YOUR_JSON_PATH ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!mkdir data
%cd data

!kaggle competitions download -c nbme-score-clinical-patient-notes
!unzip -q nbme-score-clinical-patient-notes.zip

%cd ..

!mkdir weights

!mkdir tokenizer_codes
%cd tokenizer_codes
!kaggle datasets download -d thanhns/deberta-v2-3-fast-tokenizer
!unzip deberta-v2-3-fast-tokenizer.zip
%cd ..

!mkdir deberta-tokenizer
%cd deberta-tokenizer
!kaggle datasets download -d thanhns/deberta-tokenizer
!unzip deberta-tokenizer.zip
%cd ..

In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
# The following is necessary if you want to use the fast tokenizer for deberta v2 or v3
# This must be done before importing transformers
import shutil
from pathlib import Path

transformers_path = Path("/usr/local/lib/python3.7/dist-packages/transformers")

input_dir = Path("./tokenizer_codes")

convert_file = input_dir / "convert_slow_tokenizer.py"
conversion_path = transformers_path/convert_file.name

if conversion_path.exists():
    conversion_path.unlink()

shutil.copy(convert_file, transformers_path)
deberta_v2_path = transformers_path / "models" / "deberta_v2"

for filename in ['tokenization_deberta_v2.py', 'tokenization_deberta_v2_fast.py']:
    filepath = deberta_v2_path/filename
    
    if filepath.exists():
        filepath.unlink()

    shutil.copy(input_dir/filename, filepath)

# Run

In [ ]:
!ls

In [ ]:
!python train.py

# Save

- 次のセルの`$YOUR_WEIGHT_PATH`は自分で作成した学習済みモデルを置くパス
    - e.g. `"/content/drive/MyDrive/kaggle/2022-NBME/weights/"`

In [ ]:
!zip -r model.zip ./weights/*
!cp model.zip $YOUR_WEIGHT_PATH